# Libraries

In [22]:
import os, re, xlsxwriter, string
from pathlib import Path
import pandas as pd
from collections import Counter


# Functions

In [23]:
def excel_letter(num):
    rem = int(num/26)
    return string.ascii_uppercase[rem-1]*bool(rem)+string.ascii_uppercase[num-26*rem]
    

def create_paragraph_record(number_of_paragraphs):
    div, mod = divmod(number_of_paragraphs, 500)
    df = pd.DataFrame(columns = ['File Name', 'Starting Row', 'Ending Row', 'Name of RA'])

    # Col 1: create list of 1 to div or div-1 (number of individual entry files)
    # Col 2: create list of 1, 501, ... (starting paragraph number for each entry file)
    # Col 3: create list of 500, 1000, ... (ending paragraph number for each entry file)
    # Col 4: empty col
    
    # Number of paragraphs = e.g. 1, 59, 1424, ...
    if mod > 0:
        list1 = [x + 1 for x in range(div+1)]
        list2 = [500 * x + 1 for x in range(div+1)]
        list3 = [500 * (x + 1) for x in range(div+1)]
        
        # Adjust last entry for col 3 (ending paragraph number for last entry file)
        if number_of_paragraphs > 500:
            list3[-1] = list3[-2] + mod
        else:
            list3[-1] = mod
    
    # Number of paragraphs = e.g. 0, 500, 1000, ...
    else:
        list1 = [x + 1 for x in range(div)]
        list2 = [500 * x + 1 for x in range(div)]
        list3 = [500 * (x + 1) for x in range(div)]
    
    df['File Name'] = list1
    df['Starting Row'] = list2
    df['Ending Row'] = list3
    df['Name of RA'] = ""

    return df

def add_formats(indv_entryfile):
    # Add head column formats
    left_fmt = indv_entryfile.add_format({'left': 1, 'align': 'center', 'valign' : 'vcenter'})
    right_fmt = indv_entryfile.add_format({'right': 1, 'align': 'center', 'valign' : 'vcenter'})
    left_fmt.set_text_wrap()
    right_fmt.set_text_wrap()

    # Add the merge format
    merge_format = indv_entryfile.add_format({
            'bold': 1,
            'border': 1,
            'align': 'center',
            'valign': 'vcenter'})
        
    # Add the bold format
    bold_format = indv_entryfile.add_format({"bold": 1, 'border': 1, "align": "center", "valign": "vcenter"})
    
    
    #Add default format
    default = indv_entryfile.add_format({'border': 0, 'align': 'center', 'valign': 'vcenter'})

    return bold_format, merge_format, left_fmt, right_fmt, default

        
def modify_paragraph_by_bolding_one_keyword(para_with_keywords_bolded, keyword, bold_format):
    para_with_keywords_bolded_new = []
    
    for chunk in para_with_keywords_bolded:
        # Normal string
        try:
            len(chunk)
            # Split chunk by keyword
            # The output will be a list of strings, where keywords are either removed, 
            # or converted to "" (if they start or end the paragraph). 
            # E.g. "The hurdle rate is 7%." -> ['The ', ' is 7%.']
            # E.g. "Hurdle rate is 7%." -> ['', ' is 7%.']
            # E.g. "The hurdle rate" -> ['The ', '']
            # E.g. "Hurdle rate" -> ['', '']
            chunk_splitted = re.split(keyword, chunk, flags=re.I)
            
            for index, string in enumerate(chunk_splitted):
                # Keyword appears at start or end of para_splitted
                if string == '':
                    para_with_keywords_bolded_new.append(bold_format)
                    para_with_keywords_bolded_new.append(keyword)
                # Keyword appears between 2 strings
                else:
                    para_with_keywords_bolded_new.append(string)
                    # Keyword can only appear before the last string,
                    # and keyword can't appear after the 2nd last string if the last string is ''.
                    if index < len(chunk_splitted) - 2 or (index == len(chunk_splitted) - 2 and chunk_splitted[-1] != ''):
                        para_with_keywords_bolded_new.append(bold_format)
                        para_with_keywords_bolded_new.append(keyword)                     
        # Bold format
        except:
            para_with_keywords_bolded_new.append(chunk)   

    return para_with_keywords_bolded_new
    
def get_paragraph_with_keywords_bolded(para, keyword_list, bold_format):
    para_with_keywords_bolded = [para]
    for keyword in keyword_list:
        para_with_keywords_bolded = modify_paragraph_by_bolding_one_keyword(para_with_keywords_bolded, keyword, bold_format)
    return para_with_keywords_bolded

# Import files

In [ ]:
# UPDATE new_round
# for record keeping, update most recently done round here: 
# MOST RECENT ROUND: 13
new_round = 'round13'
# import dataset of all new paragraphs with proper name
df = pd.read_csv('./new_paras/all_new_20240401.csv')
# sort with recency bias -- secondary sort by firm name
df = df.sort_values(by=['Date', 'Firm_name'], ascending=False).reset_index(drop=True)
print(f'{df.shape[0]=}')
df.head()

# Get sheet template and set up header

In [25]:
template = pd.read_excel('./data_ref/entryfile_template.xlsx', engine='openpyxl', nrows=0, header=[0,1])
header = Counter([x[0] for x in template.columns])
# convert header 1 values to xlsxwriter range strings
# i.e. 0-10 -> A1:J1
start = 0
for key in header:
    new_start = start + header[key]
    header[key] = excel_letter(start)+str(1)+':'+excel_letter(start+header[key]-1)+str(1)
    start = new_start
# get left and right ends of ranges for future formatting purposes
lefts = [x.split(':')[0][:-1] for x in list(header.values())]
rights = [x.split(':')[1][:-1] for x in list(header.values())]

header_2 = [x[1] for x in template.columns]
#convert header 2 values to tuples with their respective xlsxwriter locations
# i.e. Required -> (Required, AE2)
sub_head = []
for i, key in enumerate(header_2):
    sub_head.append((key, excel_letter(i)))
# Convert subhead to dictionary that provides associated Excel letter for relevant columns
head_d = dict((y, x) for y, x in sub_head)

# Import Keywords

In [26]:
# Import keywords
keywords = pd.read_csv('./data_ref/keywords.txt', sep = "\t", header = None)
keyword_list = list(keywords[0].str.lower())

# Hard-coded variables

In [34]:
# Column widths dictionary for each column based on template. 
# Note: there is no row-width since it gets overwritten by text-wrap
# Unmentioned columns default to standard width
col_widths = {
'Observation ID': 13,
'Keyword': 23,
'Paragraph': 123,
'Date': 24,
'gvkey': 19,
'CUSIP': 19,
'Firm_name': 40,
'Title': 44,
'Subtitle': 86,
'WACC (weighted average cost of capital)': 33,
'Hurdle rate': 14,
'Hurdle premium': 14,
'Fudge factor': 14,
'OCC (opportunity cost of capital)': 26,
'Expected return': 14,
'Require a return': 14,
'required rate of return': 19,
'discount rate': 14,
'expected rate of return': 19,
'required return': 14,
'expect a return': 14,
'weighted cost of capital': 19,
'Notes': 36,
'Report': 19,
'Country': 23,
'many_hurdles': 12,
'hq_flag': 10,
'hurdle_ma': 10,
'hurdle_forward': 14,
'Cost of capital': 14,
'Cost of debt': 14,
'Cost of equity': 14,
'many_hurdles': 12,
'hurdle_above_coc': 15,
'hurdle_stable': 14,
'hurdle_varies': 14,
'hurdle_comparison': 16,
'hurdle_1': 14,
'hurdle_2': 14,
'hurdle_3': 14,
'hurdle_4': 14,
'hurdle_5': 14,
'hurdle_subtype': 23,
'h_subtype_spec': 23,
'hurdle_1_subtype': 23,
'h1_subtype_spec': 23,
'hurdle_2_subtype': 23,
'h2_subtype_spec': 23,
'hurdle_3_subtype': 23,
'h3_subtype_spec': 23,
'hurdle_4_subtype': 23,
'h4_subtype_spec': 23,
'hurdle_5_subtype': 23,
'h5_subtype_spec': 23,
'hurdle_above_coc_reason': 23,
'hurdle_stable_reason': 23,
'hurdle_varies_reason': 23,
'hurdle_comparison_group': 23,
'hurdle_notes': 23,
}

# This dict contains the name of the columns along with their data type enforcement information
# Change dictionary to add/change accepted options, or change a data type enforcement. 
col_data_types = { 
    'Useful': {'validate': 'list', 'options': [0,1]},	
    'Flag': {'validate': 'list', 'options': [0,1]},
    'Interesting': {'validate': 'list', 'options': [0,1]},	
    'Lookup': {'validate': 'list', 'options': [0,1]},	
    'QA': {'validate': 'list', 'options': [0,1]},
    'Required': {'validate': 'decimal', 'criteria': '>', 'value': '-1000'},
    'Target': {'validate': 'decimal', 'criteria': '>', 'value': '-1000'},
    'Expected': {'validate': 'decimal', 'criteria': '>', 'value': '-1000'},	
    'Realized': {'validate': 'decimal', 'criteria': '>', 'value': '-1000'},	
    'Tax': {'validate': 'list', 'options': ['before','after']},			
    'Lever': {'validate': 'list', 'options': ['unlevered','levered']},			
    'Real': {'validate': 'list', 'options': ['real','nominal']},
    'ROIC_target': {'validate': 'list', 'options': [0,1]},
    'Cost of capital': {'validate': 'decimal', 'criteria': '>', 'value': '-1000'},	
    'WACC (weighted average cost of capital)': {'validate': 'decimal', 'criteria': '>', 'value': '-1000'},	
    'OCC (opportunity cost of capital)': {'validate': 'decimal', 'criteria': '>', 'value': '-1000'},
    'Cost of equity': {'validate': 'decimal', 'criteria': '>', 'value': '-1000'}, 
    'Cost of debt': {'validate': 'decimal', 'criteria': '>', 'value': '-1000'},
    'Hurdle rate': {'validate': 'decimal', 'criteria': '>', 'value': '-1000'},	
    'Hurdle premium': {'validate': 'decimal', 'criteria': '>', 'value': '-1000'},
    'Expected return': {'validate': 'decimal', 'criteria': '>', 'value': '-1000'},	
    'Require a return': {'validate': 'decimal', 'criteria': '>', 'value': '-1000'},	
    'required rate of return': {'validate': 'decimal', 'criteria': '>', 'value': '-1000'},	
    'discount rate': {'validate': 'decimal', 'criteria': '>', 'value': '-1000'},	
    'required return': {'validate': 'decimal', 'criteria': '>', 'value': '-1000'},
    'expect a return': {'validate': 'decimal', 'criteria': '>', 'value': '-1000'},
    'weighted cost of capital': {'validate': 'decimal', 'criteria': '>', 'value': '-1000'},
    'Fudge factor': {'validate': 'decimal', 'criteria': '>', 'value': '-1000'},
    'hq_flag': {'validate': 'list', 'options': [0,1]},	
    'hurdle_ma': {'validate': 'list', 'options': [0,1]},	
    'hurdle_forward': {'validate': 'list', 'options': [0,1]},
    'hurdle_subtype': {'validate': 'custom'},
    'h_subtype_spec': {'validate': 'custom'},
    'many_hurdles': {'validate': 'list', 'options': [0,1]},	
    'hurdle_1': {'validate': 'decimal', 'criteria': '>', 'value': '-1000'},	
    'hurdle_1_subtype': {'validate': 'custom'},
    'h1_subtype_spec': {'validate': 'custom'},	
    'hurdle_2': {'validate': 'decimal', 'criteria': '>', 'value': '-1000'},	
    'hurdle_2_subtype': {'validate': 'custom'},
    'h2_subtype_spec': {'validate': 'custom'},
    'hurdle_3': {'validate': 'decimal', 'criteria': '>', 'value': '-1000'},	
    'hurdle_3_subtype': {'validate': 'custom'},	
    'h3_subtype_spec': {'validate': 'custom'},	
    'hurdle_4': {'validate': 'decimal', 'criteria': '>', 'value': '-1000'},	
    'hurdle_4_subtype': {'validate': 'custom'},
    'h4_subtype_spec': {'validate': 'custom'},
    'hurdle_5': {'validate': 'decimal', 'criteria': '>', 'value': '-1000'},
    'hurdle_5_subtype': {'validate': 'custom'},	
    'h5_subtype_spec': {'validate': 'custom'},	
    'hurdle_above_coc': {'validate': 'list', 'options': [0,1]},	
    'hurdle_above_coc_reason': {'validate': 'custom'},
    'hurdle_stable': {'validate': 'list', 'options': [0,1]},
    'hurdle_stable_reason': {'validate': 'custom'},	
    'hurdle_varies': {'validate': 'list', 'options': [0,1]},	
    'hurdle_varies_reason': {'validate': 'custom'},
    'hurdle_comparison': {'validate': 'custom'},	
    'hurdle_comparison_group': {'validate': 'custom'},	
    'hurdle_notes': {'validate': 'custom'},	
    'Range': {'validate': 'list', 'options': [0,1]},	
    'Notes': {'validate': 'custom'},
}

# Create paragraph record

In [28]:
# Create paragraph record
number_of_paragraphs = df.shape[0]
paragraph_record = create_paragraph_record(number_of_paragraphs)

# Save paragraph record
# defend against careless naming mistakes
if os.path.exists(f'./new_paras/records/{new_round}_record.xlsx'):
    print(f'Change round name -- {new_round} already exists.')
else:
    paragraph_record.to_excel(f'./new_paras/records/{new_round}_record.xlsx', index=False)

# create round folder
if not os.path.exists(f'./new_paras/sheets/{new_round}'):
    os.mkdir(f'./new_paras/sheets/{new_round}')
else:
    print(f'Change round name -- {new_round} already exists.')

Change round name -- round12 already exists.


# Main loop

In [35]:
entryfiles_combined = df
print(entryfiles_combined.shape)

# Format: worksheet.write(row, col, value)
# For all paragraphs in entryfiles_combined (rows 2 to entryfiles_combined.max_row)
# So row goes from 1 to entryfiles_combined.max_row-1.
for row in range(entryfiles_combined.shape[0]):
    # For the 1st row of every individual entry file (1, 501, ...) 
    if (row+1) in paragraph_record['Starting Row'].tolist():
        print('made it here at row '+ str(row))
        # Create new individual entry file - workbook and worksheet
        indv_entryfile_filename = str(row // 500 + 1) + ".xlsx"
        print("Entry file:", indv_entryfile_filename)
        indv_entryfile_filepath = Path(f'./new_paras/sheets/{new_round}/{indv_entryfile_filename}')
        indv_entryfile = xlsxwriter.Workbook(indv_entryfile_filepath)
        worksheet = indv_entryfile.add_worksheet()

        # Needed row and column formats
        bold_format, merge_format, left_fmt, right_fmt, default = add_formats(indv_entryfile)

        # Write out sheet's column labels and apply formats
        #row 1 
        for head in header:
            writehead = head
            #change list checking for the future
            if 'Other' in head:
                writehead = ''
            worksheet.merge_range(header[head], writehead, merge_format) #type:ignore
        #row 2
        for head in sub_head:
            worksheet.write(head[1]+str(2), head[0], bold_format)
            worksheet.set_column(head[1]+str(':')+head[1], col_widths.get(head[0], 10))
            if head[1] in lefts:
                worksheet.set_column(head[1]+str(':')+head[1], col_widths.get(head[0], 10), left_fmt)
            elif head[1] in rights:
                worksheet.set_column(head[1]+str(':')+head[1], col_widths.get(head[0], 10), right_fmt)
            else:
                worksheet.set_column(head[1]+str(':')+head[1], col_widths.get(head[0], 10), default)

        # Iterate over tupples in sub_head and add data type enforcement and locks
        for col_name,col_letter in sub_head:
            # Check if column is in lock_column_data
            
            # Check if the column is in col_data_types
            if col_name in col_data_types.keys():  
                # get validation info of column
                val_info = col_data_types[col_name]

                # get validation type
                val_type = val_info['validate']
                
                # Error message displayed for invalid entries
                # in case of list of acceptable values
                if val_type == 'list':
                    # create error message
                    error_msg = f"Invalid entry. Please enter one of:\n{val_info['options']}."
                    # create data validation dictionary
                    data_val_dict = {'validate': val_type, 'source': val_info['options'], 
                                     'input_message': '', 'error_message': error_msg}  
                # in case of decimal values  (regular quantitative column)
                elif val_type == 'decimal': 
                    # create error message
                    error_msg = "Invalid entry. Please enter a valid number."  
                    # create data validation dictionary
                    data_val_dict = {'validate': val_type, 'criteria': val_info['criteria'],
                                     'value': val_info['value'], 'input_message': '', 
                                     'error_message': error_msg} 
                # in case of text values (regular qualitative column)
                else:
                    # create error message
                    error_msg = "Invalid entry. Please enter text."  
                    # create data validation dictionary             
                    data_val_dict = {'validate': val_type, 'value': f'=NOT(ISNUMBER({col_letter}3:{col_letter}502))', 
                                     'input_message': '', 'error_title': '', 'error_message': error_msg} 

                # This function codes the excel cells as a data validation cell. 3-502 is the max range of the entire column in excel. 
                # It is better to mention the entire column without the exact row number but at the time the code was written, this was the best way
                # Keep an eye out for improvements in the parameters as there have been suggestions. 
                # Restrict manual entry by setting 'input_message' and 'error_message'
                worksheet.data_validation(f'{col_letter}3:{col_letter}502', data_val_dict) # type: ignore

    # Get keyword and paragraph
    para = entryfiles_combined['Paragraph'].values[row]
    # Get paragraph with keywords bolded
    para_with_keywords_bolded = get_paragraph_with_keywords_bolded(para, keyword_list, bold_format) #type:ignore
    
    # Write out sheet entries
    row_indv_entryfile = row % 500 + 3
    for col in entryfiles_combined.columns:
        if col in head_d:
            if pd.isna(entryfiles_combined[col].iloc[row]):
                continue
            worksheet.write(head_d[col]+str(row_indv_entryfile), entryfiles_combined[col].iloc[row]) #type:ignore
           
    # Write out paragraph with keyword bolded
    worksheet.write_rich_string(head_d['Paragraph']+str(row_indv_entryfile), *para_with_keywords_bolded) #type:ignore
           
    # Write out paragraph number
    worksheet.write('A'+str(row_indv_entryfile), row+1) #type:ignore
    
    # Close the entry file (1-500, 501-1000, ..., <..>-max_row)
    if (row + 1) in paragraph_record['Ending Row'].tolist():
        indv_entryfile.close() #type:ignore
           
print("---")
print("Done!")

(22835, 10)
made it here at row 0
Entry file: 1.xlsx
made it here at row 500
Entry file: 2.xlsx
made it here at row 1000
Entry file: 3.xlsx
made it here at row 1500
Entry file: 4.xlsx
made it here at row 2000
Entry file: 5.xlsx
made it here at row 2500
Entry file: 6.xlsx
made it here at row 3000
Entry file: 7.xlsx
made it here at row 3500
Entry file: 8.xlsx
made it here at row 4000
Entry file: 9.xlsx
made it here at row 4500
Entry file: 10.xlsx
made it here at row 5000
Entry file: 11.xlsx
made it here at row 5500
Entry file: 12.xlsx
made it here at row 6000
Entry file: 13.xlsx
made it here at row 6500
Entry file: 14.xlsx
made it here at row 7000
Entry file: 15.xlsx
made it here at row 7500
Entry file: 16.xlsx
made it here at row 8000
Entry file: 17.xlsx
made it here at row 8500
Entry file: 18.xlsx
made it here at row 9000
Entry file: 19.xlsx
made it here at row 9500
Entry file: 20.xlsx
made it here at row 10000
Entry file: 21.xlsx
made it here at row 10500
Entry file: 22.xlsx
made it h